In [31]:
import numpy as np
import pandas as pd
import scipy.stats as st
import math
from bs4 import BeautifulSoup
import requests
def black_scholes(stock, X, exp):
    headers = {'User Agent':'Mozilla/5.0'}
    url = 'https://ycharts.com/indicators/10_year_treasury_rate'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    interest = soup.findAll('td')[10]
    i = interest.text
    rfr = i.replace('%','')
    rfr = float(rfr)
    print(f"Current Interest Rate: {rfr}")
    def getprice(stock):
        stock = stock.upper()
        url = f'https://finance.yahoo.com/quote/{stock}?p={stock}&.tsrc=fin-srch'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        Price = soup.find('span', class_="Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)")
        S = Price.text
        return S
    def getvolatility(stock):
        stock = stock.upper()
        url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = volatility.text
        return vol
        print(f"Volatility: {vol}")
    vol = getvolatility(stock)
    S = getprice(stock)
    vol = float(vol)
    S = float(S)
    print(f"Current stock price: {S}")
    print(f"Volatility: {vol*100}%")
    t = exp/365
    v = vol
    r = rfr/100
    d1 = (np.log(S/X)+(r + .5*v**2)*t) / (v*math.sqrt(t))
    d2 = d1 - v*math.sqrt(t)
    call_price = S*(st.norm.cdf(d1)) - (X/(math.exp(r*t)))*(st.norm.cdf(d2))
    call_price = float(round(call_price, 2))
    call_delta = math.exp((r-r)*t) * st.norm.cdf(d1)
    call_delta = float(round(call_delta, 2))
    put_price = call_price + X/(math.exp(r*t)) - S
    put_price = float(round(put_price, 2))
    put_delta = math.exp((r-r)*t) * (st.norm.cdf(d1)-1)
    put_delta = float(round(put_delta, 2))
    call_gamma = (math.exp((r-r)*t) * st.norm.pdf(d1)) / (S * v*math.sqrt(t))
    call_gamma = float(round(call_gamma, 2))
    put_gamma = (math.exp((r-r)*t) * st.norm.pdf(d1)) / (S * v*math.sqrt(t))
    put_gamma = float(round(call_gamma, 2))
    theta = -S *st.norm.pdf(d1)*v/(2*math.sqrt(t)) - r*X*math.exp(-r*t)*st.norm.cdf(d2)
    calltheta = theta/365
    calltheta = float(round(calltheta, 2))
    ptheta = -S *st.norm.pdf(d1)*v/(2*math.sqrt(t)) + r*X*math.exp(-r*t)*st.norm.cdf(-d2)
    puttheta = ptheta/365
    puttheta = float(round(puttheta, 2))
    call_vega = S*math.exp((r-r)*t)*st.norm.pdf(d1)*math.sqrt(t)
    cvega = call_vega/100
    cvega = float(round(cvega, 2))
    put_vega = S*math.exp((r-r)*t)*st.norm.pdf(d1)*math.sqrt(t)
    pvega = put_vega/100
    pvega = float(round(pvega, 2))
    callbreakeven = call_price + X
    putbreakeven = X - put_price
    cpop = callbreakeven
    ppop = 100 - putbreakeven
    callsnputs = {
        'Stock Price': [S],
        'Strike Price': [X],
        'Breakeven': [callbreakeven, putbreakeven],
        'Option Price': [call_price, put_price], 
        'Delta': [call_delta, put_delta], 
        'Gamma': [call_gamma, put_gamma],
        'Vega': [cvega, pvega], 
        'Theta': [calltheta, puttheta],
        
    }
    rows = "Calls", "Puts"
    data = pd.DataFrame(callsnputs, rows)
    return data

a = input("Stock ticker: ")
b = float(input("Strike price: "))
c = float(input("Time to expiration: (in days) ")) 

black_scholes(a, b, c)

Stock ticker: aapl
Strike price: 150
Time to expiration: (in days) 5
Current Interest Rate: 1.29
Current stock price: 149.1
Volatility: 24.69%


,Stock Price,Strike Price,Breakeven,Option Price,Delta,Gamma,Vega,Theta
Calls,149.1,150.0,151.32,1.32,0.43,0.09,0.07,-0.17
Puts,149.1,150.0,147.81,2.19,-0.57,0.09,0.07,-0.17
